# OD Lab 1, part A

In [120]:
import sys
import random
from pprint import pprint as pp
random.seed(42)
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [121]:
import neo4j
import py2neo
print(neo4j.__version__)
print(py2neo.__version__)

1.7.6
4.3.0


In [129]:
from neo4j import GraphDatabase

# instantiate driver
NEO4J_URI="bolt://localhost:7687"
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)

In [130]:
with gdb.session() as session:
    print(session.run(
        "call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;").data())

[{'name': 'Neo4j Kernel', 'version': '3.5.15', 'edition': 'enterprise'}]


## A.2 Loading Data


We will read data from https://dblp.uni-trier.de/. From the XML's description of data in https://dblp.org/faq/16154937.html, the following elements are represented

> - article – An article from a journal or magazine.
> - inproceedings – A paper in a conference or workshop proceedings.
> - proceedings – The proceedings volume of a conference or workshop.
> - book – An authored monograph or an edited collection of articles.
> - incollection – A part or chapter in a monograph.
> - phdthesis – A PhD thesis.
> - mastersthesis – A Master's thesis. There are only very few Master's theses in dblp.
> - www – A web page. There are only very few web pages in dblp. See also the notes on person records.

We will rely on the script provided in https://github.com/ThomHurks/dblp-to-csv, and we will be removing some of the elements by editing from the `dtd` file, because they are out of the scope of the laboratory. In particular we will be removing

- book
- incollection
- phdthesis
- masterthesis
- www

Find the modified version of the dtd file in the github repository in https://github.com/diegoquintanav/od-lab-1/blob/master/notebooks/dblp-raw/dblp_slim.dtd

## Reading XML

In [131]:
!head "dblp-raw/dblp.xml"

<?xml version="1.0" encoding="ISO-8859-1"?>
<!DOCTYPE dblp SYSTEM "dblp.dtd">
<dblp>
<article mdate="2018-01-07" key="tr/meltdown/s18" publtype="informal">
<author>Paul Kocher</author>
<author>Daniel Genkin</author>
<author>Daniel Gruss</author>
<author>Werner Haas</author>
<author>Mike Hamburg</author>
<author>Moritz Lipp</author>


In [132]:
# size of xml file
!du -h "dblp-raw/dblp.xml"

2,7G	dblp-raw/dblp.xml


### Converting XML to CSV

The script mentioned earlier is executed as

```bash
#!/bin/bash
./XMLToCSV.py --annotate --neo4j dblp-raw/dblp.xml dblp-raw/dblp_slim.dtd output_slim/output.csv --relations author:authored_by journal:published_in publisher:published_by school:submitted_at editor:edited_by cite:has_citation
```

and the resulting `neo4j-admin import` command is

```bash
#!/bin/bash
neo4j-admin import --mode=csv --database=dblp_slim.db --delimiter ";" --array-delimiter "|" --id-type INTEGER --nodes:inproceedings "output_slim/output_inproceedings_header.csv,output_slim/output_inproceedings.csv" --nodes:article "output_slim/output_article_header.csv,output_slim/output_article.csv" --nodes:proceedings "output_slim/output_proceedings_header.csv,output_slim/output_proceedings.csv" --nodes:editor "output_slim/output_editor.csv" --relationships:edited_by "output_slim/output_editor_edited_by.csv" --nodes:publisher "output_slim/output_publisher.csv" --relationships:published_by "output_slim/output_publisher_published_by.csv" --nodes:journal "output_slim/output_journal.csv" --relationships:published_in "output_slim/output_journal_published_in.csv" --nodes:author "output_slim/output_author.csv" --relationships:authored_by "output_slim/output_author_authored_by.csv" --nodes:cite "output_slim/output_cite.csv" --relationships:has_citation "output_slim/output_cite_has_citation.csv"
```



By modifying the `dtd` file, we obtained a smaller graph
- node count from 9,985,270 to 7,338, 701
- relationship count from 19,917,751 to 17,079,387

Finally, by running `call db.schema.visualization()` we get something like this

![schema1](schemas/dblp_slim_before/graph.png)


In [133]:
!ls output_slim

neo4j_import.sh		       output_inproceedings.csv
output_article.csv	       output_inproceedings_header.csv
output_article_header.csv      output_journal.csv
output_author_authored_by.csv  output_journal_published_in.csv
output_author.csv	       output_proceedings.csv
output_cite.csv		       output_proceedings_header.csv
output_cite_has_citation.csv   output_publisher.csv
output_editor.csv	       output_publisher_published_by.csv
output_editor_edited_by.csv


In [134]:
!head output_slim/output_article.csv

0;Daniel Genkin|Daniel Gruss|Michael Schwarz 0001|Mike Hamburg|Moritz Lipp|Paul Kocher|Stefan Mangard|Thomas Prescher 0002|Werner Haas|Yuval Yarom;;;;;;;;;;;https://spectreattack.com/spectre.pdf;;;meltdownattack.com;tr/meltdown/s18;2018-01-07;;;;;;;informal;;;Spectre Attacks: Exploiting Speculative Execution.;;;;;2018
1;Daniel Genkin|Daniel Gruss|Michael Schwarz 0001|Mike Hamburg|Moritz Lipp|Paul Kocher|Stefan Mangard|Thomas Prescher 0002|Werner Haas|Yuval Yarom;;;;;;;;;;;https://meltdownattack.com/meltdown.pdf;;;meltdownattack.com;tr/meltdown/m18;2018-01-07;;;;;;;informal;;;Meltdown;;;;;2018
2;Frank Manola;;;;;;;;;;;;;;GTE Laboratories Incorporated;tr/gte/TR-0263-08-94-165;2019-10-25;August;;;;;;informal;;;An Evaluation of Object-Oriented DBMS Developments: 1994 Edition.;;;db/journals/gtelab/index.html#TR-0263-08-94-165;TR-0263-08-94-165;1994
3;Michael L. Brodie|Michael Stonebraker;;;;;;;;;;;;;;GTE Laboratories Incorporated;tr/gte/TR-0222-10-92-165;2019-10-25;March;This report is also

### Missing nodes and relationships

We are then missing the following nodes

- topics
- keywords
- journals
- volumes

and the following relationships

- topic -> has -> keywords
- article -> cited_by -> article
- author -> reviews -> article


### Fake citations


citations are hard to parse from xml data, so we will be **randomly** linking articles between them using the `cited_in` relationship

- creating a relationship, using the nodes

```cypher
MATCH (a:article),(b:article)
WHERE ID(a) = 12 AND ID(b) = 13
CREATE (a)-[r:cited_by]->(b)
RETURN type(r)
```

- deleting all relationships

```cypher
MATCH p=(:article)-[r:cited_by]->(:article) DELETE r
```

- query relationships

```cypher
MATCH p=(:article)-[r:cited_by]->(:article) RETURN p LIMIT 25
```


Before creating citations, we need to fetch the IDs of existing articles. We will be creating citations only between `article` nodes.

In [135]:
q_article_ids = "MATCH (n:article) RETURN ID(n) LIMIT 10000"

with gdb.session() as session:
    article_ids = [v[0] for v in session.run(q_article_ids).values()]
    
article_ids[:5]

[2470775, 2470776, 2470777, 2470778, 2470779]

In [136]:
len(article_ids)

10000

#### Delete citations

In [137]:
# delete existing citations before inserting new ones
with gdb.session() as session:
    _out = session.run("MATCH p=(:article)-[r:cited_by]->(:article) DELETE r RETURN COUNT(r)")
    print(_out.data())

[{'COUNT(r)': 0}]


#### Create `cited_by` relationships

In [138]:
# https://neo4j.com/docs/driver-manual/1.7/sessions-transactions/#driver-transactions-transaction-functions

q_add_citation_rel_id = """MATCH (a:article),(b:article)
WHERE ID(a) = $id_a AND ID(b) = $id_b
CREATE p=(a)-[r:cited_by]->(b)
RETURN p"""


def add_citation_rel(driver, id_a, id_b):
    with driver.session() as session:
        # Caller for transactional unit of work
        return session.write_transaction(create_citation_rel, id_a, id_b)

# Simple implementation of the unit of work
def create_citation_rel(tx, id_a, id_b):
    return tx.run(q_add_citation_rel_id, id_a=id_a, id_b = id_b)

Add 500 relationships of type `cited_by`

In [139]:
random.seed(42)
# pick a sample of 500 papers, and make them be cited by other three papers at random
created_citations = []

for article in random.sample(article_ids, 500):
    for citation in random.sample(article_ids, 3):
        if article != citation: # they can't cite themselves
            _out = add_citation_rel(gdb, id_a=article, id_b=citation)
            created_citations.append(_out.data())
        else:
            print("Skipping: ",article, citation)

Skipping:  2551992 2551992


In [142]:
created_citations[0]

[{'p': <Path start=<Node id=2545142 labels={'article'} properties={'volume': '94-17', 'journal': 'Universität Trier, Mathematik/Informatik, Forschungsbericht', 'mdate': neotime.Date(2017, 6, 8), 'year': 1994, 'author': ['Anna Slobodová', 'Christoph Meinel'], 'publtype': 'informal', 'title': 'A Unifying Theoretical Background for Some BDD-based Data Structures', 'article': 401, 'key': 'tr/trier/MI94-17'}> end=<Node id=2561995 labels={'article'} properties={'ee': ['https://doi.org/10.1016/j.compchemeng.2008.08.009'], 'year': 2009, 'author': ['Alexandre C. Dimian', 'Costin Sorin Bildea', 'Klaas Steur', 'Pietro Altimari'], 'title': 'Steady-state behaviour of PFR-separation-recycle systems with simultaneous exothermic and endothermic, first-order reactions.', 'article': 7431, 'url': 'db/journals/cce/cce33.html#SteurBAD09', 'volume': '33', 'number': '3', 'journal': 'Computers & Chemical Engineering', 'pages': '628-635', 'mdate': neotime.Date(2017, 6, 5), 'author-orcid': ['0000-0001-7707-1366

#### Make articles `UNIQUE`

In [143]:
# no repeated articles
with gdb.session() as session:
    session.run("CREATE CONSTRAINT ON (n:article) ASSERT n.article IS UNIQUE")

#### Query citations

In [144]:
with gdb.session() as session:
    _out = session.run("MATCH p=(:article)-[r:cited_by]->(:article) RETURN r LIMIT 5")
    for elem in _out.data():
        print(elem)

{'r': <Relationship id=17081984 nodes=(<Node id=2470781 labels=set() properties={}>, <Node id=2554932 labels=set() properties={}>) type='cited_by' properties={}>}
{'r': <Relationship id=17081983 nodes=(<Node id=2470781 labels=set() properties={}>, <Node id=2560494 labels=set() properties={}>) type='cited_by' properties={}>}
{'r': <Relationship id=17081982 nodes=(<Node id=2470781 labels=set() properties={}>, <Node id=2559104 labels=set() properties={}>) type='cited_by' properties={}>}
{'r': <Relationship id=17081031 nodes=(<Node id=2470784 labels=set() properties={}>, <Node id=2560355 labels=set() properties={}>) type='cited_by' properties={}>}
{'r': <Relationship id=17081030 nodes=(<Node id=2470784 labels=set() properties={}>, <Node id=2552977 labels=set() properties={}>) type='cited_by' properties={}>}


### Fake keywords
It's not trivial to parse keywords and topics from data, so we will fake some topics and random keywords using the `faker` library (https://faker.readthedocs.io/en/master/) as explained in http://www.jexp.de/blog/html/create_random_data.html

In [145]:
from faker import Faker
from faker.providers import lorem

fake = Faker()
fake.seed_instance(42)
fake.add_provider(lorem)

In [147]:
# +-100 fake, non repeated keywords
fake_keywords = [
    (ix, word) for ix, word in enumerate(
        list(
            {fake.sentence(nb_words=3).rstrip('.') for _ in range(100)}
        ), start=1)]
len(fake_keywords)

100

In [148]:
fake_keywords[:10]

[(1, 'Partner rest measure'),
 (2, 'Former reflect even'),
 (3, 'Ball character him'),
 (4, 'Career left anyone'),
 (5, 'Each'),
 (6, 'Machine dream key'),
 (7, 'Option production base'),
 (8, 'Remember'),
 (9, 'Knowledge city technology'),
 (10, 'Build movie several')]

In [149]:
# delete keywords
with gdb.session() as session:
    _out = session.run("MATCH (n:keyword) DELETE n RETURN COUNT(n)")
    print(_out.data())

[{'COUNT(n)': 0}]


In [150]:
q_create_keyword = "CREATE (n:keyword {id:$id, keyword:$keyword}) RETURN n"

created_keywords = []

with gdb.session() as session:
    for ix, keyword in fake_keywords:
        _out = session.run(q_create_keyword, id=ix, keyword=keyword)
        created_keywords.append(_out.data())

In [151]:
created_keywords[0]

[{'n': <Node id=7340970 labels={'keyword'} properties={'keyword': 'Partner rest measure', 'id': 1}>}]

In [152]:
# create UNIQUE constraint
with gdb.session() as session:
    session.run("CREATE CONSTRAINT ON (n:keyword) ASSERT n.keyword IS UNIQUE")

#### Assign 5 keywords to 1000 articles, at random

In [158]:
# delete relationships
with gdb.session() as session:
    _out = session.run("MATCH p=(:article)-[r:has_keyword]->(:keyword) DELETE r RETURN COUNT(r)")
    print(_out.data())

[{'COUNT(r)': 0}]


In [159]:
# assign 5 keywords randomly to 1000 articles
random.seed(42)

q_add_keywords = """MATCH (a:article),(b:keyword)
WHERE ID(a) = $article_id AND b.keyword = $keyword
CREATE p=(a)-[r:has_keyword]->(b)
RETURN p"""

created_kw_rel = []

with gdb.session() as session:
    for article_id in random.sample(article_ids, 1000):
        for ix, keyword in random.sample(fake_keywords, 5):
            _out = session.run(q_add_keywords, article_id=article_id, keyword=keyword)
            created_kw_rel.append(_out.data())

In [160]:
created_kw_rel[0]

[{'p': <Path start=<Node id=2545142 labels={'article'} properties={'volume': '94-17', 'journal': 'Universität Trier, Mathematik/Informatik, Forschungsbericht', 'mdate': neotime.Date(2017, 6, 8), 'year': 1994, 'author': ['Anna Slobodová', 'Christoph Meinel'], 'publtype': 'informal', 'title': 'A Unifying Theoretical Background for Some BDD-based Data Structures', 'article': 401, 'key': 'tr/trier/MI94-17'}> end=<Node id=7341069 labels={'keyword'} properties={'keyword': 'Deep something future', 'id': 81}> size=1>}]

In [161]:
with gdb.session() as session:
    _out = session.run("MATCH p=(:article)-[r:has_keyword]->(:keyword) RETURN p LIMIT 5")
    print(_out.data()[0])

{'p': <Path start=<Node id=2534363 labels={'article'} properties={'ee': ['https://doi.org/10.1007/s13222-016-0230-9'], 'year': 2016, 'author': ['Theo Härder'], 'title': 'Prof. Dr. Dr. h.\u202fc. Hans-Jürgen Appelrath.', 'article': 101838, 'url': 'db/journals/dbsk/dbsk16.html#Harder16', 'volume': '16', 'number': '3', 'journal': 'Datenbank-Spektrum', 'pages': '259-260', 'mdate': neotime.Date(2018, 6, 26), 'ee-type': ['oa'], 'key': 'journals/dbsk/Harder16'}> end=<Node id=7340970 labels={'keyword'} properties={'keyword': 'Partner rest measure', 'id': 1}> size=1>}


### Faking reviewers 

We will create a `reviewed_by` relationship between authors and articles

In [162]:
# delete relationships
with gdb.session() as session:
    _out = session.run("MATCH p=(:article)-[r:reviewed_by]->(:author) DELETE r RETURN COUNT(r)")
    print(_out.data())

[{'COUNT(r)': 0}]


In [163]:
q_author_ids = "MATCH (n:author) RETURN ID(n) LIMIT 10000"

with gdb.session() as session:
    author_ids = [v[0] for v in session.run(q_author_ids).values()]
    
author_ids[:5]

[4779298, 4779299, 4779300, 4779301, 4779302]

In [164]:
# assign between 3 and 4 reviewers randomly to 1000 articles
random.seed(42)

q_add_reviewers = """MATCH (a:article),(b:author)
WHERE ID(a) = $article_id AND ID(b) = $author_id
CREATE p=(a)-[r:reviewed_by]->(b)
RETURN p"""

created_review_rel = []

with gdb.session() as session:
    for article_id in random.sample(article_ids, 1000):
        for author_id in random.sample(author_ids, 3):
            _out = session.run(q_add_reviewers, article_id=article_id, author_id=author_id)
            created_review_rel.append(_out.data())

In [165]:
created_review_rel[0]

[{'p': <Path start=<Node id=2545142 labels={'article'} properties={'volume': '94-17', 'journal': 'Universität Trier, Mathematik/Informatik, Forschungsbericht', 'mdate': neotime.Date(2017, 6, 8), 'year': 1994, 'author': ['Anna Slobodová', 'Christoph Meinel'], 'publtype': 'informal', 'title': 'A Unifying Theoretical Background for Some BDD-based Data Structures', 'article': 401, 'key': 'tr/trier/MI94-17'}> end=<Node id=4832453 labels={'author'} properties={'author': 'Navin Kartik'}> size=1>}]

## A.3 Evolving the graph 



### Store the review and the approval sent by each reviewer

Since the queries in part B don't need this information, we can store the `review_decision` and the `review_text` as _edge_ attributes of the relation `REVIEWED_BY`.


Alternatively, and in the case these reviews were a requirement in part B, they can be implemented by creating a `Review` node, and linking it to articles, in a four step process.

1. Create a `Review` article for each `Article` that has reviewers (i.e. is connected to authors by `reviewed_by` edges). Each `Review` node will have an attribute named `review_contents´.
2. Attach review to the article
3. Attach reviewers to the review instance
4. Optional, delete the `reviewed_by` edges from the step 1.

In [166]:
with gdb.session() as session:
    out = session.run("MATCH p=(:article)-[r:reviewed_by]->(:author) return ID(r)")
    rel_ids = [r[0] for r in out.values()]
    
rel_ids[:5]

[17088467, 17088466, 17088465, 17087513, 17087512]

In [167]:
q_merge_review_attributes = """MATCH p=(:article)-[r:reviewed_by]->(:author)
WHERE ID(r) = $rel_id
SET r = {review_accepted: $review_accepted, review_text: $review_text}
RETURN r
"""

created_review_rel_attr = []

random.seed(42)

with gdb.session() as session:
    for rel_id in rel_ids:
        _out = session.run(q_merge_review_attributes, 
                    rel_id=rel_id,
                    review_accepted=fake.pybool(), 
                    review_text=fake.texts(nb_texts=1, max_nb_chars=500)[0])
        created_review_rel_attr.append(_out.data())

In [168]:
created_review_rel_attr[0]

[{'r': <Relationship id=17088467 nodes=(<Node id=2470781 labels=set() properties={}>, <Node id=4835571 labels=set() properties={}>) type='reviewed_by' properties={'review_text': 'Continue anything wait local state husband. Officer rather charge specific. Be easy newspaper indicate other.\nSimply herself training father open. Figure perform participant science way debate. Enough ball dream necessary choose. Late order fact discuss religious reflect.\nReach under skin person. Interesting name positive training step. Arrive society organization station.\nBuy read record wall matter management. Our threat same page.\nDirector purpose team onto.', 'review_accepted': True}>}]

### Store a reviewing policy for each Journal or Conference

This can also be implemented as an `review_policy_min_count` to the `Journal` and `Proceedings` labels. We will set all of them to 3.

In [169]:
with gdb.session() as session:
    journal_ids = session.run("MERGE (n:journal) SET n.review_policy_min_count = 3 return n").values()
    proceeding_ids = session.run("MERGE (n:proceedings) SET n.review_policy_min_count = 3 return n").values()

In [170]:
journal_ids[:5]

[[<Node id=4738120 labels={'journal'} properties={'review_policy_min_count': 3, 'journal': 'meltdownattack.com'}>],
 [<Node id=4738121 labels={'journal'} properties={'review_policy_min_count': 3, 'journal': 'GTE Laboratories Incorporated'}>],
 [<Node id=4738122 labels={'journal'} properties={'review_policy_min_count': 3, 'journal': 'University of California at Berkeley'}>],
 [<Node id=4738123 labels={'journal'} properties={'review_policy_min_count': 3, 'journal': 'ANSI X3H2'}>],
 [<Node id=4738124 labels={'journal'} properties={'review_policy_min_count': 3, 'journal': 'ANSI X2H2'}>]]

In [171]:
proceeding_ids[:2]

[[<Node id=4685034 labels={'proceedings'} properties={'ee': ['https://doi.org/10.1007/978-3-319-11698-3'], 'editor': ['Barbara Carminati', 'C.-C. Jay Kuo', 'Man Ho Au'], 'year': 2014, 'isbn': ['978-3-319-11697-6'], 'editor-orcid': ['0000-0002-7502-4731', '0000-0003-2068-9530'], 'series-href': ['db/series/lncs/index.html'], 'title': "Network and System Security - 8th International Conference, NSS 2014, Xi'an, China, October 15-17, 2014, Proceedings", 'url': 'db/conf/nss/nss2014.html', 'review_policy_min_count': 3, 'volume': '8792', 'mdate': neotime.Date(2019, 5, 14), 'series': ['Lecture Notes in Computer Science'], 'publisher': ['Springer'], 'proceedings': 3546997, 'booktitle': 'NSS', 'key': 'conf/nss/2014'}>],
 [<Node id=4685035 labels={'proceedings'} properties={'ee': ['https://doi.org/10.1007/978-3-642-38631-2'], 'editor': ['Javier López 0001', 'Ravi Sandhu', 'Xinyi Huang'], 'year': 2013, 'isbn': ['978-3-642-38630-5'], 'series-href': ['db/series/lncs/index.html'], 'title': 'Network a

### Display the affiliation of the author to an organization or a company 

This can be solved by creating an `affiliation_institution_type` attribute, and also an `affiliation_institution_name`. Note that we could have created these as nodes, but the queries from part B do not refer to this information.


In [172]:
q_set_affiliations = """MATCH (n:author)
WHERE ID(n) = $author_id
SET n = {affiliation_institution_type: $affiliation_institution_type, affiliation_institution_name: $affiliation_institution_name}
RETURN n
"""

institution_types = ['University', 'Company', 'NGO']
created_affiliations = []

random.seed(42)

with gdb.session() as session:
    for author_id in author_ids:
        _out = session.run(q_set_affiliations, 
                    author_id=author_id,
                    affiliation_institution_name=fake.company(), 
                    affiliation_institution_type=random.choice(institution_types))
        
        created_affiliations.append(_out.values())
        

## Final graph

![schemasafter](schemas/dblp_slim_after/graph.svg)